# Connect to ES

In [1]:
import elasticsearch
from elasticsearch import Elasticsearch
from decouple import config
import urllib3
urllib3.disable_warnings()

In [2]:
elasticsearch.__version__

(8, 3, 3)

In [3]:
client = Elasticsearch(
    config('ES_URL'),
    basic_auth=("elastic", config('ES_PASS')),
    verify_certs=False,
)

/home/reissant/miniforge3/envs/py10/lib/python3.10/site-packages/elasticsearch/_sync/client/__init__.py:395: SecurityWarning: Connecting to 'https://ml-elasticsearch.kcp.bbp.epfl.ch:443' using TLS with verify_certs=False is insecure
  _transport = transport_class(


In [4]:
client.info()

ObjectApiResponse({'name': 'elasticsearch-coordinating-0', 'cluster_name': 'elastic', 'cluster_uuid': 'dv5oiXnCRJiB8nhV7a81RA', 'version': {'number': '8.3.3', 'build_flavor': 'default', 'build_type': 'tar', 'build_hash': '801fed82df74dbe537f89b71b098ccaff88d2c56', 'build_date': '2022-07-23T19:30:09.227964828Z', 'build_snapshot': False, 'lucene_version': '9.2.0', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'})

# Explore tables
tables names are articles and paragraphs

In [6]:
import json

In [7]:
# articles index mapping
resp = dict(client.indices.get(index="articles"))
print(json.dumps(resp, indent=4))

{
    "articles": {
        "aliases": {},
        "mappings": {
            "dynamic": "strict",
            "properties": {
                "abstract": {
                    "type": "text"
                },
                "article_id": {
                    "type": "keyword"
                },
                "authors": {
                    "type": "text"
                },
                "doi": {
                    "type": "keyword"
                },
                "is_english": {
                    "type": "boolean"
                },
                "journal": {
                    "type": "keyword"
                },
                "license": {
                    "type": "keyword"
                },
                "pmc_id": {
                    "type": "keyword"
                },
                "publish_time": {
                    "type": "date",
                    "format": "yyyy-MM-dd"
                },
                "pubmed_id": {
                    "type":

In [8]:
# paragraphs index mapping
resp = dict(client.indices.get(index="paragraphs"))
print(json.dumps(resp, indent=4))

{
    "paragraphs": {
        "aliases": {},
        "mappings": {
            "dynamic": "strict",
            "properties": {
                "article_id": {
                    "type": "keyword"
                },
                "embedding": {
                    "type": "dense_vector",
                    "dims": 384,
                    "index": true,
                    "similarity": "dot_product"
                },
                "is_bad": {
                    "type": "boolean"
                },
                "paragraph_id": {
                    "type": "short"
                },
                "section_name": {
                    "type": "keyword"
                },
                "text": {
                    "type": "text"
                }
            }
        },
        "settings": {
            "index": {
                "routing": {
                    "allocation": {
                        "include": {
                            "_tier_preference": "data_con

In [12]:
query = {
    "match": {
        "text": "Fourier Transform Spectroscopy"
    }
}
resp = client.search(index="paragraphs", query=query, size=1)
print("Got %d Hits:" % resp['hits']['total']['value'])
for hit in resp['hits']['hits']:
    print(json.dumps(hit["_source"], indent=4))

Got 68 Hits:
{
    "article_id": "f619d10de9834bbb8b0252c9e8455489",
    "section_name": "",
    "text": "This study reports on the cellular uptake of folate tethered micelles using a branched skeleton of poly(ethylene glycol) and poly(\u03b5-caprolactone). The chemical structures of the copolymers were characterized by proton nuclear magnetic resonance spectroscopy, and Fourier transform infrared spectroscopy. Doxorubicin (DOX) was utilized as an anticancer drug. The highest drug loading efficiencies of DOX in the folate decorated micelle (DMCF) and folate-free micelle (DMC) were found to be 88.5 % and 88.2 %, respectively, depending on the segment length of the poly(\u03b5-caprolactone) in the copolymers. A comparison of fluorescent microscopic images of the endocytosis pathway in two cells lines, human breast cancer cells (MCF-7) and human oral cavity carcinoma cells (KB), revealed that the micelles were engulfed by KB and MCF-7 cells following in vitro incubation for 1 h. Flow cyto

In [21]:
resp = client.get(index="articles", id="f619d10de9834bbb8b0252c9e8455489")
print(json.dumps(resp['_source'], indent=4))

{
    "article_id": "f619d10de9834bbb8b0252c9e8455489",
    "authors": "Fourth Annual Conference of the American Society for Nanomedicine",
    "title": "Howard E. Gendelman, Lajos P. Balogh, Raj Bawa, Michelle Bradbury, Esther H. Chang, Wah Chiu, Omid Farokhzad, Marianna Foldvari, Gregory Lanza, Kuan Wang",
    "abstract": "",
    "pubmed_id": "24573530",
    "pmc_id": "3951957",
    "doi": "10.1007/s11481-014-9526-4"
}
